# Visualizing Tokens

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/labmlai/inspectus/blob/main/notebooks/token_viz.ipynb)

This Jupyter notebook demonstrates how to use the Inspectus library to visualize tokens and any related data.

In [ ]:
!pip install inspectus
!pip install transformers

In [2]:
from transformers import AutoTokenizer, AutoConfig, GPT2LMHeadModel
import inspectus
import torch

Following cell sets up a GPT-2 model from Huggingface's Transformers library. It initializes the tokenizer and model configuration, then creates the GPT-2 model.

In [ ]:
context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

This cell takes a sentence and tokenizes it using the previously initialized tokenizer. The tokenized output is then used to create input IDs for the model and a list of tokens. It uses the 'offset_mapping' attribute returned by the tokenizer to slice the original text into individual tokens.

In [4]:
text= 'Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a line in section 1.10.32. The standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et Malorum" by Cicero are also reproduced in their exact original form, accompanied by English versions from the 1914 translation by H. Rackham.'
tokenized = tokenizer(
    text,
    return_tensors='pt',
    return_offsets_mapping=True
)
input_ids = tokenized['input_ids']

tokens = [text[s: e] for s, e in tokenized['offset_mapping'][0]]

In [ ]:
with torch.no_grad():
    res = model(input_ids=input_ids.to(model.device), output_attentions=True)

Following cell gets the top 5 predictions for each token in the given text along with the loss.

In [ ]:
from torch.nn.functional import nll_loss

logits = res['logits']
losses = []
entropies = []
token_info = []
for i in range(len(tokens)):
    loss = nll_loss(logits[0, i], input_ids[0, i])
    losses.append(loss.item())
    
    entropy = -torch.sum(torch.nn.functional.softmax(logits[0, i]) * torch.nn.functional.log_softmax(logits[0, i]))
    entropies.append(entropy.item())

    pred_token_indices = torch.argsort(logits[0, i])[:5]
    pred_tokens = [tokenizer.decode([idx]) for idx in pred_token_indices]
    token_info.append(f"pred 1: {pred_tokens[0]}\npred 2: {pred_tokens[1]}\npred 3: {pred_tokens[2]}\npred 4: {pred_tokens[3]}\npred 5: {pred_tokens[4]}")

The `inspectus.tokens` function visualizes tokens using their losses. Hover on to a token to view aditional info.

In [ ]:
inspectus.tokens(tokens, {"loss": losses, "entropy": entropies}, token_info=token_info, theme="light")